# Small_Scale_Analysis.ipynb
This notebook:
* Obtains values for the various automatic metrics (WER, CER, etc.) versus the Ground Truth for both Google ASR and WhisperX.
* Writes the results to a df.
* Aggregates the results across the entire df, the scripted podcasts, and the non-scripted podcasts.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import re
import string

# torch metrics
import torch
from torcheval.metrics.functional import word_error_rate
from torcheval.metrics import BLEUScore
from torcheval.metrics.text import WordInformationLost
from torcheval.metrics import WordInformationPreserved

# huggingface metrics
from datasets import load_metric
import evaluate

# set up objects for object-oriented metrics
bleu_score = BLEUScore(n_gram=4)
wil_score = WordInformationLost()
wip_score = WordInformationPreserved()
cer_score = evaluate.load("cer")
bert_score = evaluate.load("bertscore")
rouge_score = evaluate.load('rouge')

/home/grads/m/mariateleki/anaconda3/envs/pytorch_env_3.8/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("./csv/df-english-fisher-annotations-test-all3.csv")

In [3]:
df["length_GroundTruth"] = -1.0
df["length_GoogleASR"] = -1.0
df["length_WhisperX"] = -1.0

df["wer_GoogleASR"] = -1.0
df["wer_WhisperX"] = -1.0

df["cer_GoogleASR"] = -1.0
df["cer_WhisperX"] = -1.0

df["wil_GoogleASR"] = -1.0
df["wil_WhisperX"] = -1.0

df["wip_GoogleASR"] = -1.0
df["wip_WhisperX"] = -1.0

df["bleu_GoogleASR"] = -1.0
df["bleu_WhisperX"] = -1.0

df["bert_p_GoogleASR"] = -1.0
df["bert_p_WhisperX"] = -1.0
df["bert_r_GoogleASR"] = -1.0
df["bert_r_WhisperX"] = -1.0
df["bert_f_GoogleASR"] = -1.0
df["bert_f_WhisperX"] = -1.0

df["rouge1_GoogleASR"] = -1.0
df["rouge1_WhisperX"] = -1.0
df["rouge2_GoogleASR"] = -1.0
df["rouge2_WhisperX"] = -1.0
df["rougeL_GoogleASR"] = -1.0
df["rougeL_WhisperX"] = -1.0

def count_punctuation(input_string):
    
    # get punctuation string
    p = string.punctuation
    p = p.replace("'","")
    p = p.replace("[","")
    p = p.replace("]","")
    
    count = 0
    for char in input_string:
        if char in p:
            count += 1
    return count

for index, row in df.iterrows():
    
    # transcript length
    for asr in ["WhisperX", "GoogleASR", "GroundTruth"]:
        if row[f"rpl_{asr}"] != "":
            num_words = len(row[f"rpl_{asr}"].split(" "))
        else:
            num_words = 0
        df.loc[index, f"length_{asr}"] = num_words
    
    # specific token counts
    for target in ["uh", "um", "well"]:
        for asr in ["WhisperX", "GoogleASR", "GroundTruth"]:
            df[f"{target}_count_{asr}"] = -1
            df[f"{target}_count_{asr}"] = df[f"rpl_{asr}"].str.count(r"\b"+re.escape(target)+r"\b", flags=re.IGNORECASE)
            
    # english-fisher-annotator parse counts
    for target in ["INTJ", "PRN", "EDITED"]:
        for asr in ["WhisperX", "GoogleASR", "GroundTruth-max", "GroundTruth-min", "GroundTruth-neutral"]:
            df[f"{target}_count_{asr}"] = -1
            df[f"{target}_count_{asr}"] = df[f"{asr}_parse"].str.count(r"\("+re.escape(target)+r" ", flags=re.IGNORECASE)
    
    # WER
    df.loc[index, "wer_GoogleASR"] = word_error_rate(input=row["rpl_GoogleASR"], target=row["rpl_GroundTruth"]).item()
    df.loc[index, "wer_WhisperX"] = word_error_rate(input=row["rpl_WhisperX"], target=row["rpl_GroundTruth"]).item()
    
    # CER
    df.loc[index, "cer_GoogleASR"] = cer_score.compute(predictions=[row["rpl_GoogleASR"]], references=[row["rpl_GroundTruth"]])
    df.loc[index, "cer_WhisperX"] = cer_score.compute(predictions=[row["rpl_WhisperX"]], references=[row["rpl_GroundTruth"]])
    
     # WIL
    wil_score.update([row["rpl_GoogleASR"]], [row["rpl_GroundTruth"]])
    df.loc[index, "wil_GoogleASR"] = wil_score.compute().item()
    
    wil_score.update([row["rpl_WhisperX"]], [row["rpl_GroundTruth"]])
    df.loc[index, "wil_WhisperX"] = wil_score.compute().item()
    
    # WIP
    wip_score.update([row["rpl_GoogleASR"]], [row["rpl_GroundTruth"]])
    df.loc[index, "wip_GoogleASR"] = wip_score.compute().item()
    
    wip_score.update([row["rpl_WhisperX"]], [row["rpl_GroundTruth"]])
    df.loc[index, "wip_WhisperX"] = wip_score.compute().item()
    
    # BLEU
    bleu_score.update([row["rpl_GoogleASR"]], [row["rpl_GroundTruth"]])
    df.loc[index, "bleu_GoogleASR"] = bleu_score.compute().item()
    
    bleu_score.update([row["rpl_WhisperX"]], [row["rpl_GroundTruth"]])
    df.loc[index, "bleu_WhisperX"] = bleu_score.compute().item()
    
    # BERTscore
    google = bert_score.compute(predictions=[row["rpl_GoogleASR"]], references=[row["rpl_GroundTruth"]], lang="en")
    df.loc[index, "bert_p_GoogleASR"] = google["precision"]
    df.loc[index, "bert_r_GoogleASR"] = google["recall"]
    df.loc[index, "bert_f_GoogleASR"] = google["f1"]
    
    whisper = bert_score.compute(predictions=[row["rpl_WhisperX"]], references=[row["rpl_GroundTruth"]], lang="en")
    df.loc[index, "bert_p_WhisperX"] = whisper["precision"]
    df.loc[index, "bert_r_WhisperX"] = whisper["recall"]
    df.loc[index, "bert_f_WhisperX"] = whisper["f1"]
    
    # rouge
    google = rouge_score.compute(predictions=[row["rpl_GoogleASR"]], references=[row["rpl_GroundTruth"]])
    df.loc[index, "rouge1_GoogleASR"] = google["rouge1"]
    df.loc[index, "rouge2_GoogleASR"] = google["rouge2"]
    df.loc[index, "rougeL_GoogleASR"] = google["rougeL"]
    
    whisper = rouge_score.compute(predictions=[row["rpl_WhisperX"]], references=[row["rpl_GroundTruth"]])
    df.loc[index, "rouge1_WhisperX"] = whisper["rouge1"]
    df.loc[index, "rouge2_WhisperX"] = whisper["rouge2"]
    df.loc[index, "rougeL_WhisperX"] = whisper["rougeL"]
    
    # punctuation counts
    df["punctuation_count_GoogleASR"] = df["GoogleASR"].apply(count_punctuation)
    df["punctuation_count_WhisperX"] = df["WhisperX"].apply(count_punctuation)
    df["punctuation_count_GroundTruth-max"] = df["GroundTruth-max"].apply(count_punctuation)
    df["punctuation_count_GroundTruth-min"] = df["GroundTruth-min"].apply(count_punctuation)
    df["punctuation_count_GroundTruth-neutral"] = df["GroundTruth-neutral"].apply(count_punctuation)

display(df)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,Unnamed: 0.1,Unnamed: 0,nums,ids,labels,percent_diff,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix,GroundTruth,GroundTruth-max,GroundTruth-neutral,GroundTruth-min,GoogleASR,WhisperX,rpl_GroundTruth,rpl_GoogleASR,rpl_WhisperX,GroundTruth-max_parse,GroundTruth-max_orig_dys,GroundTruth-max_dys,GroundTruth-min_parse,GroundTruth-min_orig_dys,GroundTruth-min_dys,GroundTruth-neutral_parse,GroundTruth-neutral_orig_dys,GroundTruth-neutral_dys,GoogleASR_parse,GoogleASR_orig_dys,GoogleASR_dys,WhisperX_parse,WhisperX_orig_dys,WhisperX_dys,length_GroundTruth,length_GoogleASR,length_WhisperX,wer_GoogleASR,wer_WhisperX,cer_GoogleASR,cer_WhisperX,wil_GoogleASR,wil_WhisperX,wip_GoogleASR,wip_WhisperX,bleu_GoogleASR,bleu_WhisperX,bert_p_GoogleASR,bert_p_WhisperX,bert_r_GoogleASR,bert_r_WhisperX,bert_f_GoogleASR,bert_f_WhisperX,rouge1_GoogleASR,rouge1_WhisperX,rouge2_GoogleASR,rouge2_WhisperX,rougeL_GoogleASR,rougeL_WhisperX,uh_count_WhisperX,uh_count_GoogleASR,uh_count_GroundTruth,um_count_WhisperX,um_count_GoogleASR,um_count_GroundTruth,well_count_WhisperX,well_count_GoogleASR,well_count_GroundTruth,INTJ_count_WhisperX,INTJ_count_GoogleASR,INTJ_count_GroundTruth-max,INTJ_count_GroundTruth-min,INTJ_count_GroundTruth-neutral,PRN_count_WhisperX,PRN_count_GoogleASR,PRN_count_GroundTruth-max,PRN_count_GroundTruth-min,PRN_count_GroundTruth-neutral,EDITED_count_WhisperX,EDITED_count_GoogleASR,EDITED_count_GroundTruth-max,EDITED_count_GroundTruth-min,EDITED_count_GroundTruth-neutral,punctuation_count_GoogleASR,punctuation_count_WhisperX,punctuation_count_GroundTruth-max,punctuation_count_GroundTruth-min,punctuation_count_GroundTruth-neutral
0,1,1,1,1ll5WGWjWANfGKHZfrpL5A,not-scripted,14.20,spotify:show:5HLsz7WFjW8hzJurMDdozi,Granger Smith Podcast,"American, Texan, father, husband, musician, Gr...",Granger Smith,['en'],https://anchor.fm/s/11170a28/podcast/rss,spotify:episode:1ll5WGWjWANfGKHZfrpL5A,How do you move on from THIS?,"Episode 21: On this week's episode, I answered...",25.891600,show_5HLsz7WFjW8hzJurMDdozi,1ll5WGWjWANfGKHZfrpL5A,if you're ever in deep deep grief and you wond...,"If you're ever in deep, deep grief, and you wo...","If you're ever in deep, deep grief and you won...","If you're ever in deep, deep grief and you won...",If you're ever in deep deep grief and you wond...,"If you're ever in deep, deep grief and you won...",if you're ever in deep deep grief and you wond...,if you're ever in deep deep grief and you wond...,if you're ever in deep deep grief and you wond...,(S (SBAR (UNK if) (S (S (NP (UNK you)) (VP (UN...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,(S (SBAR (UNK if) (S (S (NP (UNK you)) (VP (UN...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,(S (SBAR (UNK if) (S (S (NP (UNK you)) (VP (UN...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,(S (SBAR (UNK if) (S (S (NP (UNK you)) (VP (UN...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,(S (SBAR (UNK if) (S (S (NP (UNK you)) (VP (UN...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,if _ you _ 're _ ever _ in _ deep E deep _ gri...,267.0,260.0,255.0,0.089888,0.108614,0.058296,0.068012,0.149395,0.158718,0.850605,0.841282,0.818645,0.809326,0.983228,0.982010,0.969828,0.967203,0.976482,0.974550,0.933810,0.927798,0.868941,0.862319,0.933810,0.927798,0,0,3,0,0,3,0,0,0,1,1,7,7,6,0,1,0,1,0,2,8,8,7,8,22,42,68,53,60
1,2,2,2,1wHRMsWVurmo56xd1AcSfe,scripted,1.24,spotify:show:1g056e2x0Y9AwW6CQF3qA5,Mythology,Myths endure for a reason. This episodic audio...,Parcast Network,['en'],https://feeds.megaphone.fm/mythology,spotify:episode:1wHRMsWVurmo56xd1AcSfe,Orphan Boy & Elk Dog Pt. 1,Long Arrow is an orphan and an outcast—unable ...,36.144767,show_1g056e2x0Y9AwW6

## Describe overall df

In [4]:
temp_df = df.describe()

for col in temp_df.columns:
    if "count" not in col and "duration" not in col and "percent_diff" not in col and "length" not in col and "nums" not in col:
        temp_df[col] = round(temp_df[col]*100.0,2)
    else:
        temp_df[col] = round(temp_df[col],2)
        
display(temp_df)

,Unnamed: 0.1,Unnamed: 0,nums,percent_diff,duration,length_GroundTruth,length_GoogleASR,length_WhisperX,wer_GoogleASR,wer_WhisperX,cer_GoogleASR,cer_WhisperX,wil_GoogleASR,wil_WhisperX,wip_GoogleASR,wip_WhisperX,bleu_GoogleASR,bleu_WhisperX,bert_p_GoogleASR,bert_p_WhisperX,bert_r_GoogleASR,bert_r_WhisperX,bert_f_GoogleASR,bert_f_WhisperX,rouge1_GoogleASR,rouge1_WhisperX,rouge2_GoogleASR,rouge2_WhisperX,rougeL_GoogleASR,rougeL_WhisperX,uh_count_WhisperX,uh_count_GoogleASR,uh_count_GroundTruth,um_count_WhisperX,um_count_GoogleASR,um_count_GroundTruth,well_count_WhisperX,well_count_GoogleASR,well_count_GroundTruth,INTJ_count_WhisperX,INTJ_count_GoogleASR,INTJ_count_GroundTruth-max,INTJ_count_GroundTruth-min,INTJ_count_GroundTruth-neutral,PRN_count_WhisperX,PRN_count_GoogleASR,PRN_count_GroundTruth-max,PRN_count_GroundTruth-min,PRN_count_GroundTruth-neutral,EDITED_count_WhisperX,EDITED_count_GoogleASR,EDITED_count_GroundTruth-max,EDITED_count_GroundTruth-min,EDITED_count_GroundTruth-neutral,punctuation_count_GoogleASR,punctuation_count_WhisperX,punctuation_count_GroundTruth-max,punctuation_count_GroundTruth-min,punctuation_count_GroundTruth-neutral
count,1000.00,1000.00,10.00,10.00,10.00,10.00,10.00,10.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,10.00,10.0,10.0,10.00,10.0,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00
mean,550.00,550.00,5.50,10.63,29.48,294.50,290.80,291.80,10.74,7.19,6.71,4.38,15.02,14.79,84.98,85.21,84.95,85.32,97.03,98.19,96.65,97.92,96.84,98.05,92.57,95.15,86.27,90.98,91.82,94.97,0.20,0.0,1.0,0.40,0.0,0.80,0.30,0.30,0.30,5.00,4.20,5.90,6.00,5.60,1.30,1.30,1.30,1.30,1.30,2.50,3.80,3.30,3.40,3.60,20.90,38.00,59.90,51.20,52.80
std,302.77,302.77,3.03,6.68,19.36,34.82,32.97,34.84,6.18,5.21,5.37,3.64,1.09,1.73,1.09,1.73,1.18,1.78,1.61,1.11,2.14,1.51,1.86,1.30,3.57,3.11,5.75,5.45,4.39,3.27,0.63,0.0,1.7,0.70,0.0,1.14,0.48,0.48,0.48,6.04,4.85,6.38,6.48,6.90,2.11,2.45,2.21,1.89,1.95,2.68,2.94,4.47,3.78,3.86,5.38,12.66,16.97,13.96,15.96
min,100.00,100.00,1.00,1.24,3.21,258.00,259.00,255.00,3.72,2.23,1.36,0.88,12.77,10.52,83.23,82.99,81.86,80.93,93.37,95.77,91.31,94.59,92.33,95.18,83.89,88.24,72.73,80.13,80.87,87.92,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,13.00,24.00,40.00,38.00,32.00
25%,325.00,325.00,3.25,6.30,19.21,269.00,269.50,267.25,7.36,2.93,3.56,1.43,14.63,14.44,84.46,84.34,84.86,85.07,96.36,97.67,96.16,97.06,96.15,97.48,91.34,93.67,85.21,87.32,91.09,92.91,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,1.00,1.00,1.25,1.25,1.25,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.25,0.25,18.50,30.25,48.75,40.50,45.00
50%,550.00,550.00,5.50,9.70,24.19,292.50,284.00,291.50,9.84,5.47,5.98,4.45,14.97,14.96,85.03,85.04,85.10,85.46,97.36,98.38,97.09,98.42,97.33,98.37,93.59,96.00,87.09,93.04,93.17,96.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,3.00,2.50,4.00,4.00,3.50,0.00,0.50,0.50,1.00,0.50,2.00,3.50,0.50,2.00,2.50,21.50,37.00,55.00,49.00,48.50
75%,775.00,775.00,7.75,13.45,36.12,303.50,304.50,305.00,10.80,10.53,7.70,5.25,15.54,15.66,85.37,85.56,85.50,86.18,98.05,98.95,97.77,99.06,97.83,99.00,94.17,97.50,89.38,95.24,94.06,97.50,0.00,0.0,1.0,0.75,0.0,1.75,0.75,0.75,0.75,6.50,4.00,7.75,7.75,5.75,1.75,1.00,1.00,1.00,1.75,4.50,5.75,7.00,6.25,6.25,22.00,40.00,67.00,56.00,58.50
max,1000.00,1000.00,10.00,25.67,63.90,377.00,369.00,372.00,26.35,18.24,20.58,12.88,16.77,17.01,87.23,89.48,86.04,87.80,98.94,99.48,98.99,99.39,98.96,99.43,97.13,98.38,93.88,96.93,96.77,98.38,2.00,0.0,5.0,2.00,0.0,3.00,1.00,1.00,1.00,19.00,14.00,21.00,21.00,23.00,6.00,8.00,7.00,6.00,6.00,7.00,8.00,12.00,11.00,11.00,33.00,70.00,99.00,85.00,89.00


## Describe non-scripted df

In [5]:
temp_df = df[df["labels"]=="not-scripted"].describe()

for col in temp_df.columns:
    if "count" not in col and "duration" not in col and "percent_diff" not in col and "length" not in col and "nums" not in col:
        temp_df[col] = round(temp_df[col]*100.0,2)
    else:
        temp_df[col] = round(temp_df[col],2)
        
display(temp_df)

,Unnamed: 0.1,Unnamed: 0,nums,percent_diff,duration,length_GroundTruth,length_GoogleASR,length_WhisperX,wer_GoogleASR,wer_WhisperX,cer_GoogleASR,cer_WhisperX,wil_GoogleASR,wil_WhisperX,wip_GoogleASR,wip_WhisperX,bleu_GoogleASR,bleu_WhisperX,bert_p_GoogleASR,bert_p_WhisperX,bert_r_GoogleASR,bert_r_WhisperX,bert_f_GoogleASR,bert_f_WhisperX,rouge1_GoogleASR,rouge1_WhisperX,rouge2_GoogleASR,rouge2_WhisperX,rougeL_GoogleASR,rougeL_WhisperX,uh_count_WhisperX,uh_count_GoogleASR,uh_count_GroundTruth,um_count_WhisperX,um_count_GoogleASR,um_count_GroundTruth,well_count_WhisperX,well_count_GoogleASR,well_count_GroundTruth,INTJ_count_WhisperX,INTJ_count_GoogleASR,INTJ_count_GroundTruth-max,INTJ_count_GroundTruth-min,INTJ_count_GroundTruth-neutral,PRN_count_WhisperX,PRN_count_GoogleASR,PRN_count_GroundTruth-max,PRN_count_GroundTruth-min,PRN_count_GroundTruth-neutral,EDITED_count_WhisperX,EDITED_count_GoogleASR,EDITED_count_GroundTruth-max,EDITED_count_GroundTruth-min,EDITED_count_GroundTruth-neutral,punctuation_count_GoogleASR,punctuation_count_WhisperX,punctuation_count_GroundTruth-max,punctuation_count_GroundTruth-min,punctuation_count_GroundTruth-neutral
count,600.00,600.00,6.00,6.00,6.00,6.00,6.00,6.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,600.00,6.00,6.0,6.00,6.00,6.0,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00
mean,533.33,533.33,5.33,14.24,37.03,302.00,294.67,298.00,12.98,9.74,8.87,6.05,15.03,15.32,84.97,84.68,84.85,84.71,96.67,97.67,95.91,97.13,96.29,97.40,91.61,93.64,84.32,88.07,90.48,93.34,0.33,0.0,1.67,0.67,0.0,1.33,0.50,0.50,0.50,7.83,6.33,9.17,9.33,8.67,2.17,2.17,2.00,2.00,2.00,3.67,5.33,5.33,5.17,5.50,22.83,38.67,67.67,57.17,60.50
std,314.11,314.11,3.14,5.94,19.47,42.69,41.12,43.05,6.96,5.32,5.95,3.77,0.67,0.97,0.67,0.97,1.56,1.96,1.79,1.09,2.38,1.46,2.07,1.25,4.02,3.16,6.31,5.22,5.06,3.29,0.82,0.0,1.97,0.82,0.0,1.21,0.55,0.55,0.55,6.40,5.32,6.40,6.47,7.55,2.40,2.93,2.68,2.19,2.28,2.73,2.50,4.84,3.97,3.94,5.78,16.54,17.76,15.24,15.86
min,100.00,100.00,1.00,9.05,19.08,258.00,259.00,255.00,6.98,2.71,4.18,1.53,14.22,14.29,83.94,82.99,81.86,80.93,93.37,95.77,91.31,94.59,92.33,95.18,83.89,88.24,72.73,80.13,80.87,87.92,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.00,1.00,3.00,3.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,15.00,24.00,47.00,39.00,45.00
25%,325.00,325.00,3.25,10.87,21.19,274.25,264.50,265.75,9.41,6.69,5.90,4.85,14.63,14.85,84.61,84.35,84.78,84.76,96.36,97.31,95.83,96.76,96.11,97.16,91.34,92.99,83.18,86.50,89.80,92.71,0.00,0.0,0.25,0.00,0.0,0.25,0.00,0.00,0.00,4.25,3.25,5.50,5.50,4.25,0.25,1.00,0.25,1.00,0.25,2.00,3.50,1.00,2.50,3.25,22.00,29.50,58.00,51.50,50.25
50%,550.00,550.00,5.50,13.10,30.98,296.00,286.00,297.00,10.75,10.21,7.15,5.14,14.89,14.96,85.11,85.04,85.26,85.16,97.14,97.87,96.78,97.22,97.01,97.50,93.00,93.71,86.38,87.33,92.45,93.03,0.00,0.0,1.00,0.50,0.0,1.50,0.50,0.50,0.50,6.00,4.00,7.50,7.50,5.50,1.50,1.00,1.00,1.00,1.50,3.50,5.50,6.00,5.50,5.50,22.00,33.50,66.00,55.00,57.00
75%,775.00,775.00,7.75,14.10,52.23,312.50,304.50,308.00,13.25,11.21,8.28,6.44,15.39,15.65,85.37,85.15,85.86,85.74,97.76,98.46,97.15,98.20,97.52,98.25,93.77,95.31,87.19,90.92,93.28,95.18,0.00,0.0,2.50,1.00,0.0,2.00,1.00,1.00,1.00,10.00,10.00,10.25,11.00,9.75,3.50,1.75,2.50,2.50,2.75,5.75,7.50,8.00,7.00,7.75,22.75,40.50,71.00,57.75,64.50
max,900.00,900.00,9.00,25.67,63.90,377.00,369.00,372.00,26.35,18.24,20.58,12.88,16.06,17.01,85.78,85.71,86.04,86.46,98.32,98.72,97.84,98.58,97.89,98.65,94.99,97.57,90.88,95.31,94.62,97.57,2.00,0.0,5.00,2.00,0.0,3.00,1.00,1.00,1.00,19.00,14.00,21.00,21.00,23.00,6.00,8.00,7.00,6.00,6.00,7.00,8.00,12.00,11.00,11.00,33.00,70.00,99.00,85.00,89.00


## Describe scripted df

In [6]:
temp_df = df[df["labels"]=="scripted"].describe()

for col in temp_df.columns:
    if "count" not in col and "duration" not in col and "percent_diff" not in col and "length" not in col and "nums" not in col:
        temp_df[col] = round(temp_df[col]*100.0,2)
    else:
        temp_df[col] = round(temp_df[col],2)
        
display(temp_df)

,Unnamed: 0.1,Unnamed: 0,nums,percent_diff,duration,length_GroundTruth,length_GoogleASR,length_WhisperX,wer_GoogleASR,wer_WhisperX,cer_GoogleASR,cer_WhisperX,wil_GoogleASR,wil_WhisperX,wip_GoogleASR,wip_WhisperX,bleu_GoogleASR,bleu_WhisperX,bert_p_GoogleASR,bert_p_WhisperX,bert_r_GoogleASR,bert_r_WhisperX,bert_f_GoogleASR,bert_f_WhisperX,rouge1_GoogleASR,rouge1_WhisperX,rouge2_GoogleASR,rouge2_WhisperX,rougeL_GoogleASR,rougeL_WhisperX,uh_count_WhisperX,uh_count_GoogleASR,uh_count_GroundTruth,um_count_WhisperX,um_count_GoogleASR,um_count_GroundTruth,well_count_WhisperX,well_count_GoogleASR,well_count_GroundTruth,INTJ_count_WhisperX,INTJ_count_GoogleASR,INTJ_count_GroundTruth-max,INTJ_count_GroundTruth-min,INTJ_count_GroundTruth-neutral,PRN_count_WhisperX,PRN_count_GoogleASR,PRN_count_GroundTruth-max,PRN_count_GroundTruth-min,PRN_count_GroundTruth-neutral,EDITED_count_WhisperX,EDITED_count_GoogleASR,EDITED_count_GroundTruth-max,EDITED_count_GroundTruth-min,EDITED_count_GroundTruth-neutral,punctuation_count_GoogleASR,punctuation_count_WhisperX,punctuation_count_GroundTruth-max,punctuation_count_GroundTruth-min,punctuation_count_GroundTruth-neutral
count,400.0,400.0,4.00,4.00,4.00,4.00,4.00,4.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.00,4.00,4.00,4.00,4.00,4.0,4.0,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00
mean,575.0,575.0,5.75,5.22,18.15,283.25,285.00,282.50,7.39,3.36,3.46,1.87,15.02,14.01,84.98,85.99,85.09,86.24,97.57,98.96,97.74,99.10,97.66,99.03,94.00,97.41,89.19,95.34,93.83,97.41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.75,1.00,1.00,1.00,1.00,0.0,0.0,0.25,0.25,0.25,0.75,1.50,0.25,0.75,0.75,18.00,37.00,48.25,42.25,41.25
std,330.4,330.4,3.30,3.12,14.38,17.86,19.17,18.98,2.99,1.37,2.07,1.49,1.67,2.45,1.67,2.45,0.32,1.12,1.34,0.62,1.28,0.46,1.31,0.53,2.59,0.93,3.72,1.22,2.46,0.93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.96,0.82,0.82,0.82,0.82,0.0,0.0,0.50,0.50,0.50,1.50,1.91,0.50,0.96,0.96,3.56,4.76,6.02,4.43,6.99
min,200.0,200.0,2.00,1.24,3.21,269.00,269.00,267.00,3.72,2.23,1.36,0.88,12.77,10.52,83.23,83.81,84.83,85.28,95.77,98.08,96.03,98.43,95.90,98.25,90.81,96.19,84.99,93.99,90.81,96.19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,13.00,30.00,40.00,38.00,32.00
25%,425.0,425.0,4.25,4.29,8.88,269.00,270.50,267.75,5.86,2.37,2.48,1.06,14.45,13.36,84.16,84.69,84.93,85.52,97.06,98.79,97.18,99.02,97.12,98.90,93.05,97.01,87.28,94.78,92.81,97.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.75,0.75,0.75,0.75,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,16.75,36.00,46.00,38.75,38.00
50%,550.0,550.0,5.50,5.38,16.63,279.00,280.50,278.00,7.54,3.01,3.10,1.26,15.27,14.66,84.73,85.34,84.98,85.94,97.79,99.14,97.98,99.30,97.88,99.22,94.03,97.54,88.93,95.23,93.87,97.54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50,1.00,1.00,1.00,1.00,0.0,0.0,0.00,0.00,0.00,0.00,1.00,0.00,0.50,0.50,19.00,39.00,49.50,42.00,42.50
75%,700.0,700.0,7.00,6.30,25.90,293.25,295.00,292.75,9.07,4.00,4.09,2.06,15.84,15.31,85.55,86.64,85.14,86.66,98.30,99.32,98.54,99.38,98.42,99.35,94.98,97.94,90.84,95.79,94.89,97.94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.25,1.25,1.25,1.25,1.25,0.0,0.0,0.25,0.25,0.25,0.75,2.50,0.25,1.25,1.25,20.25,40.00,51.75,45.50,45.75
max,1000.0,1000.0,10.00,8.87,36.14,306.00,310.00,307.00,10.78,5.20,6.29,4.08,16.77,16.19,87.23,89.48,85.56,87.80,98.94,99.48,98.99,99.39,98.96,99.43,97.13,98.38,93.88,96.93,96.77,98.38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.00,2.00,2.00,2.00,2.00,0.0,0.0,1.00,1.00,1.00,3.00,4.00,1.00,2.00,2.00,21.00,40.00,54.00,47.00,48.00
